In [2]:
import pandas as pd
import os

# Preparing the data.

In [3]:
data_root = os.path.join("..","Data")
sales_df = pd.read_excel(os.path.join(data_root, 'DEMAND_FILE.xlsx'), sheet_name='Daily CGD Volume - Ahmedabad')
sales_df = sales_df.iloc[0:3].T.reset_index(drop=True)
sales_df.columns = ['Date','Weekday','Sales']
sales_df = sales_df.drop(index = 0).reset_index(drop=True)
sales_df['Weekday'] = sales_df['Weekday'].apply(lambda x: 0 if x in ['Sunday', 'Saturday'] else 1)
sales_df['Date'] = pd.to_datetime(sales_df['Date'])
sales_df['Sales'] = sales_df['Sales'].astype('float64')

sales_df['Sales_MA_7'] = sales_df['Sales'].rolling(window=7).mean().shift(1)  # 7-day moving avg
sales_df

,Date,Weekday,Sales,Sales_MA_7
0,2023-04-01,0,489076.113688,NaN
1,2023-04-02,0,491107.744591,NaN
2,2023-04-03,1,463311.399700,NaN
3,2023-04-04,1,456306.343200,NaN
4,2023-04-05,1,493338.999138,NaN
...,...,...,...,...
270,2023-12-27,1,571390.999704,581611.016546
271,2023-12-28,1,576890.976666,579815.388893
272,2023-12-29,1,572104.814138,578344.850415
273,2023-12-30,0,585843.459244,576290.337907


In [4]:
import calendar
def days_in_month(row):
    return calendar.monthrange(row['Year'], row['Month'])[1]

sales_df['Day'] = sales_df['Date'].dt.day
sales_df['Weekday'] = sales_df['Date'].dt.weekday
sales_df['Month'] = sales_df['Date'].dt.month
sales_df['Year'] = sales_df['Date'].dt.year
sales_df['DayOfYear'] = sales_df['Date'].dt.dayofyear
sales_df['IsWeekend'] = (sales_df['Weekday'] >= 5).astype(int)
sales_df['DaysInMonth'] = sales_df.apply(days_in_month, axis=1)
sales_df['Date'] = sales_df['Date'].dt.date
sales_df

,Date,Weekday,Sales,Sales_MA_7,Day,Month,Year,DayOfYear,IsWeekend,DaysInMonth
0,2023-04-01,5,489076.113688,NaN,1,4,2023,91,1,30
1,2023-04-02,6,491107.744591,NaN,2,4,2023,92,1,30
2,2023-04-03,0,463311.399700,NaN,3,4,2023,93,0,30
3,2023-04-04,1,456306.343200,NaN,4,4,2023,94,0,30
4,2023-04-05,2,493338.999138,NaN,5,4,2023,95,0,30
...,...,...,...,...,...,...,...,...,...,...
270,2023-12-27,2,571390.999704,581611.016546,27,12,2023,361,0,31
271,2023-12-28,3,576890.976666,579815.388893,28,12,2023,362,0,31
272,2023-12-29,4,572104.814138,578344.850415,29,12,2023,363,0,31
273,2023-12-30,5,585843.459244,576290.337907,30,12,2023,364,1,31


In [5]:
import numpy as np
# Apply Cyclical Encoding
sales_df['Day_sin'] = np.sin(2 * np.pi * sales_df['Day'] / sales_df['DaysInMonth'])
sales_df['Day_cos'] = np.cos(2 * np.pi * sales_df['Day'] / sales_df['DaysInMonth'])

sales_df['Month_sin'] = np.sin(2 * np.pi * sales_df['Month'] / 12)
sales_df['Month_cos'] = np.cos(2 * np.pi * sales_df['Month'] / 12)

sales_df['Weekday_sin'] = np.sin(2 * np.pi * sales_df['Weekday'] / 7)
sales_df['Weekday_cos'] = np.cos(2 * np.pi * sales_df['Weekday'] / 7)

sales_df['DayOfYear_sin'] = np.sin(2 * np.pi * sales_df['DayOfYear'] / 365)
sales_df['DayOfYear_cos'] = np.cos(2 * np.pi * sales_df['DayOfYear'] / 365)

# Drop raw columns if necessary
sales_df = sales_df.drop(columns=['Weekday', 'Month', 'DayOfYear', 'DaysInMonth','Day','Year'])
sales_df

,Date,Sales,Sales_MA_7,IsWeekend,Day_sin,Day_cos,Month_sin,Month_cos,Weekday_sin,Weekday_cos,DayOfYear_sin,DayOfYear_cos
0,2023-04-01,489076.113688,NaN,1,2.079117e-01,0.978148,8.660254e-01,-0.5,-0.974928,-0.222521,9.999907e-01,0.004304
1,2023-04-02,491107.744591,NaN,1,4.067366e-01,0.913545,8.660254e-01,-0.5,-0.781831,0.623490,9.999167e-01,-0.012910
2,2023-04-03,463311.399700,NaN,0,5.877853e-01,0.809017,8.660254e-01,-0.5,0.000000,1.000000,9.995463e-01,-0.030120
3,2023-04-04,456306.343200,NaN,0,7.431448e-01,0.669131,8.660254e-01,-0.5,0.781831,0.623490,9.988797e-01,-0.047321
4,2023-04-05,493338.999138,NaN,0,8.660254e-01,0.500000,8.660254e-01,-0.5,0.974928,-0.222521,9.979172e-01,-0.064508
...,...,...,...,...,...,...,...,...,...,...,...,...
270,2023-12-27,571390.999704,581611.016546,0,-7.247928e-01,0.688967,-2.449294e-16,1.0,0.974928,-0.222521,-6.880243e-02,0.997630
271,2023-12-28,576890.976666,579815.388893,0,-5.712682e-01,0.820763,-2.449294e-16,1.0,0.433884,-0.900969,-5.161967e-02,0.998667
272,2023-12-29,572104.814138,578344.850415,0,-3.943559e-01,0.918958,-2.449294e-16,1.0,-0.433884,-0.900969,-3.442161e-02,0.999407
273,2023-12-30,585843.459244,576290.337907,1,-2.012985e-01,0.979530,-2.449294e-16,1.0,-0.974928,-0.222521,-1.721336e-02,0.999852


In [6]:
weather_df = pd.read_csv(os.path.join(data_root,'Ahmedabad_Weather_Data_Cleaned.csv')).drop(range(0,7)).reset_index(drop=True)
weather_df

,Date,Temp,Humidity,Rain
0,2023-04-08,88.9,29.8,0.0
1,2023-04-09,89.9,29.1,0.0
2,2023-04-10,91.4,28.9,0.0
3,2023-04-11,92.0,30.7,0.0
4,2023-04-12,90.0,33.3,0.0
...,...,...,...,...
263,2023-12-27,71.2,72.4,0.0
264,2023-12-28,72.5,60.0,0.0
265,2023-12-29,71.4,64.1,0.0
266,2023-12-30,71.9,69.3,0.0


In [8]:
combined_df = pd.concat([sales_df,weather_df.iloc[:,1:]],axis = 1)
combined_df = combined_df[['Date','IsWeekend', 'Day_sin', 'Day_cos', 'Month_sin', 'Month_cos', 'Weekday_sin',
                           'Weekday_cos', 'DayOfYear_sin', 'DayOfYear_cos','Temp','Humidity','Rain','Sales_MA_7',
                           'Sales']]
combined_df = combined_df.dropna().reset_index(drop=True)
combined_df

,Date,IsWeekend,Day_sin,Day_cos,Month_sin,Month_cos,Weekday_sin,Weekday_cos,DayOfYear_sin,DayOfYear_cos,Temp,Humidity,Rain,Sales_MA_7,Sales
0,2023-04-08,1,0.994522,-0.104528,8.660254e-01,-0.5,-0.974928,-0.222521,0.993257,-0.115935,90.5,33.2,0.0,470962.212977,542228.194163
1,2023-04-09,1,0.951057,-0.309017,8.660254e-01,-0.5,-0.781831,0.623490,0.991114,-0.133015,90.6,34.9,0.0,478555.367331,478717.394779
2,2023-04-10,0,0.866025,-0.500000,8.660254e-01,-0.5,0.000000,1.000000,0.988678,-0.150055,91.8,33.6,0.0,476785.317357,483230.277816
3,2023-04-11,0,0.743145,-0.669131,8.660254e-01,-0.5,0.781831,0.623490,0.985948,-0.167052,92.2,27.8,0.0,479630.871374,485970.141540
4,2023-04-12,0,0.587785,-0.809017,8.660254e-01,-0.5,0.974928,-0.222521,0.982927,-0.183998,92.4,31.1,0.0,483868.556851,484442.664286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,2023-12-20,0,-0.790776,-0.612106,-2.449294e-16,1.0,0.974928,-0.222521,-0.188227,0.982126,71.2,72.4,0.0,592911.729495,583960.393272
257,2023-12-21,0,-0.897805,-0.440394,-2.449294e-16,1.0,0.433884,-0.900969,-0.171293,0.985220,72.5,60.0,0.0,590479.026607,587184.746016
258,2023-12-22,0,-0.968077,-0.250653,-2.449294e-16,1.0,-0.433884,-0.900969,-0.154309,0.988023,71.4,64.1,0.0,586173.878188,586486.401692
259,2023-12-23,1,-0.998717,-0.050649,-2.449294e-16,1.0,-0.974928,-0.222521,-0.137279,0.990532,71.9,69.3,0.0,584772.816217,565619.814480


# Training the Model.

In [9]:

import plotly.graph_objects as go

def get_train_val_test_dfs(combined_df):
    train_size = int(0.7 * len(combined_df))  # 70% for training
    val_size = int(0.85 * len(combined_df)) # 15% for validation
    train_df  = combined_df[:train_size].reset_index(drop=True)
    val_df = combined_df[train_size:val_size].reset_index(drop=True)
    test_df = combined_df[val_size:].reset_index(drop=True)
    return train_df, val_df, test_df

def plot_training_graphs(history):
    history_df = pd.DataFrame.from_dict(history.history)
    history_df['epoch'] = history_df.index + 1

    fig = go.Figure()
    fig.add_trace(go.Line(x=history_df['epoch'], y = history_df['loss'], mode = 'lines+markers', name='Training Loss'))
    fig.add_trace(go.Line(x=history_df['epoch'], y = history_df['val_loss'], mode = 'lines+markers', name = 'Validation Loss'))
    fig.update_layout(title="Training and Validation Loss",
                    xaxis_title="Epochs", yaxis_title="Loss")
    fig.show()

    fig = go.Figure()
    fig.add_trace(go.Line(x=history_df['epoch'], y = history_df['mape'], mode = 'lines+markers', name='Training MAPE'))
    fig.add_trace(go.Line(x=history_df['epoch'], y = history_df['val_mape'], mode = 'lines+markers', name = 'Validation MAPE'))
    fig.update_layout(title="Training and Validation MAPE",
                    xaxis_title="Epochs", yaxis_title="MAPE")
    fig.show()

def plot_test_graphs(test_df):
    fig = go.Figure()

    fig.add_trace(go.Line(x = test_df['Date'],
                        y = test_df['Sales'],
                        mode = 'lines+markers',
                        name = 'Actual Sales'))
    fig.add_trace(go.Line(x = test_df['Date'],
                        y = test_df['Predicted_Sales'],
                        mode = 'lines+markers',
                        name = 'Predicted Sales'))

    fig.update_layout(xaxis_title = 'Date', yaxis_title = 'Sales')
    fig.show()



In [34]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.losses import Huber
from tensorflow.keras.callbacks import EarlyStopping

def mape(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    # Avoid division by zero by adding a small epsilon where y_true is zero
    epsilon = tf.keras.backend.epsilon()
    y_true = tf.where(tf.equal(y_true, 0), epsilon, y_true)

    return tf.reduce_mean(tf.abs((y_true - y_pred) / y_true)) * 100

def model_train(train_df, val_df, features, target):
    
    X_train, y_train = train_df[features].values.astype('float'), train_df[target].values.astype('float')
    X_val, y_val = val_df[features].values.astype('float'), val_df[target].values.astype('float')
    
    early_stopping = EarlyStopping(
    monitor='val_mape',
    mode = 'min',
    patience=15,
    restore_best_weights=True )

    # Define FFNN model
    model = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1)
    ])

    # Compile and train
    model.compile(optimizer='adam', loss='mse' , metrics=[mape])
    history = model.fit(X_train, y_train, epochs=100, batch_size=4, validation_data=(X_val, y_val),callbacks=[early_stopping])

    return model, history


In [20]:
import numpy as np

def model_test(model, test_df, val_df, features, target):
    test_df = pd.concat([val_df[-7:],test_df]).reset_index(drop=True)
    test_df['Sales_MA_7'] = np.nan
    test_df['Predicted_Sales'] = np.nan

    for row in test_df.itertuples(index=True, name='Rows'):
        if row.Index >= 7:
            test_df.loc[row.Index, 'Sales_MA_7'] = test_df.iloc[row.Index-7:row.Index]['Sales'].mean()
            X_test = np.array(test_df.loc[row.Index, features].values, dtype = np.float64)
            test_df.loc[row.Index, 'Predicted_Sales'] = model.predict(X_test.reshape(1,len(features)))
    
    return test_df

In [ ]:
features = ['IsWeekend', 'Day_sin', 'Day_cos', 'Month_sin', 'Month_cos', 'Weekday_sin',
            'Weekday_cos', 'DayOfYear_sin', 'DayOfYear_cos','Sales_MA_7',
            ]

target = 'Sales'
plot_graphs = False

train_df, val_df, test_df = get_train_val_test_dfs(combined_df)
model, history = model_train(train_df, val_df, features, target)
if(plot_graphs):
    plot_training_graphs(history)

test_df = model_test(model, test_df, val_df, features, target)
print(f"MAPE for test data: {mape(test_df.iloc[7:]['Sales'],test_df.iloc[7:]['Predicted_Sales'])}")

if(plot_graphs):
    plot_test_graphs(test_df)


Epoch 1/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 236741165056.0000 - mape: 89.4873 - val_loss: 66908479488.0000 - val_mape: 44.6663
Epoch 2/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 38035632128.0000 - mape: 34.3511 - val_loss: 2174303488.0000 - val_mape: 7.6311
Epoch 3/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1450115072.0000 - mape: 7.5842 - val_loss: 2086838272.0000 - val_mape: 6.3133
Epoch 4/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 947164032.0000 - mape: 7.5166 - val_loss: 1988514048.0000 - val_mape: 6.3727
Epoch 5/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 985361792.0000 - mape: 7.3853 - val_loss: 1989351808.0000 - val_mape: 6.3716
Epoch 6/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1286893824.0000 - mape: 7.7079 - val_loss: 2041865600.0000 - val_mape: 6.3247
Epoch 7/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 868043264.0000 - mape: 6.4590 - val_loss: 2032539008.0000 - val_mape: 6.3303
Epoch 8/100
46/46 ━━━━━━━━━━━━━━━━━━━

c:\Users\milan\Desktop\CDIS Project\.venv\Lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━

c:\Users\milan\Desktop\CDIS Project\.venv\Lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [26]:
test_df

,Date,IsWeekend,Day_sin,Day_cos,Month_sin,Month_cos,Weekday_sin,Weekday_cos,DayOfYear_sin,DayOfYear_cos,Temp,Humidity,Rain,Sales_MA_7,Sales,Predicted_Sales
0,2023-11-08,0,9.945219e-01,-0.104528,-5.000000e-01,0.866025,0.974928,-0.222521,-0.790946,0.611886,74.6,51.8,0.000,NaN,591771.403670,NaN
1,2023-11-09,0,9.510565e-01,-0.309017,-5.000000e-01,0.866025,0.433884,-0.900969,-0.780296,0.625411,75.7,48.9,0.000,NaN,607158.435735,NaN
2,2023-11-10,0,8.660254e-01,-0.500000,-5.000000e-01,0.866025,-0.433884,-0.900969,-0.769415,0.638749,74.9,50.9,0.000,NaN,587221.237680,NaN
3,2023-11-11,1,7.431448e-01,-0.669131,-5.000000e-01,0.866025,-0.974928,-0.222521,-0.758306,0.651899,75.2,53.3,0.000,NaN,548673.244632,NaN
4,2023-11-12,1,5.877853e-01,-0.809017,-5.000000e-01,0.866025,-0.781831,0.623490,-0.746972,0.664855,76.4,56.8,0.000,NaN,444510.082892,NaN
5,2023-11-13,0,4.067366e-01,-0.913545,-5.000000e-01,0.866025,0.000000,1.000000,-0.735417,0.677615,78.1,57.6,0.000,NaN,437882.345445,NaN
6,2023-11-14,0,2.079117e-01,-0.978148,-5.000000e-01,0.866025,0.781831,0.623490,-0.723644,0.690173,78.3,55.3,0.000,NaN,448402.566024,NaN
7,2023-11-15,0,5.665539e-16,-1.000000,-5.000000e-01,0.866025,0.974928,-0.222521,-0.711657,0.702527,75.7,57.2,0.000,523659.902297,457435.032133,526936.56250
8,2023-11-16,0,-2.079117e-01,-0.978148,-5.000000e-01,0.866025,0.433884,-0.900969,-0.699458,0.714673,73.9,54.1,0.000,504468.992077,471428.896804,507625.62500
9,2023-11-17,0,-4.067366e-01,-0.913545,-5.000000e-01,0.866025,-0.433884,-0.900969,-0.687053,0.726608,75.2,47.4,0.000,485079.057944,475409.127696,488114.40625
